In [28]:
import numpy as np
from enum import Enum
from queue import PriorityQueue

In [29]:
grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])
grid = np.array([
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
])

start = (1, 0)
goal = (7, 8)

In [30]:

class Action(Enum):
    # Assign the cost of each action as the third element in the tuple
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    UR_DIAG = (-1, 1, 1.41)
    UL_DIAG = (-1, -1, 1.41)
    DR_DIAG = (1, 1, 1.41)
    DL_DIAG = (1, -1, 1.41)

    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
        elif self == self.UR_DIAG:
            return '/^'
        elif self == self.UL_DIAG:
            return '\\^'
        elif self == self.DR_DIAG:
            return '\\v'
        elif self == self.DL_DIAG:
            return '/v'
        
    # Assign a new property that returns the cost of an action
    @property
    def cost(self):
        return self.value[2]
    
    # Assign a property that returns the action itself
    @property
    def delta(self):
        return (self.value[0], self.value[1])

In [31]:
# Reurns the valid moves depending on the current node
def valid(grid, current):
  valid = []
  for act in Action:
    delta = act.value
    next_pos = (current[0] + delta[0], current[1] + delta[1])

    is_inside_grid = (next_pos[0] >= 0 and next_pos[0] < len(grid)) and (next_pos[1] >= 0 and next_pos[1] < len(grid[0])) 
    
    if is_inside_grid and grid[next_pos] != 1:
      valid.append(act)
      
  return valid

In [32]:
# UNIFORM COST
def UC(grid, start, goal):
    q = PriorityQueue()
    q.put((0, start))

    visited = set()
    visited.add(start)

    branch = {}
    i=0 
    while True:
        current_node = q.get()
        current_cost = current_node[0]
        current_pos = current_node[1]
        
        if current_pos == goal:
            break

        for action in valid(grid, current_pos):
            i+=1
            delta = action.delta

            next_pos = (current_pos[0] + delta[0], current_pos[1] + delta[1])
            next_cost = current_cost + action.cost

            if next_pos not in visited:                
                visited.add(next_pos)                                   # 1. Mark it as visited           
                q.put((next_cost, next_pos))                            # 2. Add it to the queue
                branch[next_pos] = (current_pos, action, next_cost)     # 3. Add how to get there to branch

            elif next_pos in branch and branch[next_pos][2] > next_cost:  # if cost is less, update it
                branch[next_pos] = (current_pos, action, next_cost)
    
    # Retrace path
    path = []
    n = goal
    tot_cost = branch[n][2]
    
    while branch[n][0] != start:
        # append each new node to the path
        path.append(branch[n][1])
        n = branch[n][0]

    # append the start location
    path.append(branch[n][1])
    print(i)
    # reverse the path
    return path[::-1], tot_cost

In [33]:
path, cost = UC(grid, start, goal)

396


In [34]:
# PRINT RESULT
print(grid)
for act in path:
    print(act, end=' ')
print()
print(cost)

[[0 1 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 1 0 1 0 0]
 [0 0 0 1 0 0 0 1 0 1 0 0]
 [0 1 0 0 0 0 0 1 0 1 0 0]
 [0 1 0 0 0 0 0 1 1 1 0 0]
 [0 1 0 1 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0]]
v \v /^ /^ > > > > > /^ \v v v v v v /v < 
20.46
